In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn.functional import mse_loss
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
cell_data = pd.read_csv("../data/")

In [ ]:
class dataset_from_pd(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return self.dataframe.shape[0]

    def __getitem__(self, idx):
        return self.dataframe.iloc[idx,:]

In [12]:
class MLP(torch.nn.Module):
    def __init__(self, h_sizes):
        super().__init__()
        self.hidden = nn.ModuleList()
        for k in range(len(h_sizes)-1):
            self.hidden.append(nn.Linear(h_sizes[k], h_sizes[k+1]))
            self.hidden.append(nn.ReLU())

    def forward(self, x):
        for lay in self.hidden:
            x = lay(x)
        return x

In [13]:
model = MLP([8,8])
model.fit(X_test, y_test)


AttributeError: 'MLP' object has no attribute 'fit'